In [33]:
import ccxt
import pandas as pd
import ta

import logging
import time
from datetime import datetime, timedelta

class CryptoTradingBot:
    def __init__(self, exchange_id, symbols, initial_balance=1000, test_mode=True):
        self.exchange = getattr(ccxt, exchange_id)({
            'enableRateLimit': True,
            'test': test_mode
        })
        
        self.symbols = symbols
        self.initial_balance = initial_balance
        self.current_balance = initial_balance
        self.total_profit = 0  # Toplam kar takibi için değişken
        self.open_positions = {}
        self.commission_rate = 0.001  # Binde 1 komisyon
        self.max_open_positions = 5  # Maksimum 5 açık işlem
        self.fixed_trade_amount = 100  # Her işlem 100 dolar
        self.position_timeout = 900  # 15 dakika (900 saniye)
        self.stop_loss_percentage = 0.35  # %0.35 stop loss
        self.take_profit_percentage = 0.5  # %0.5 kar alma

        logging.basicConfig(
            filename='trading_log.txt',
            level=logging.INFO,
            format='%(asctime)s - %(message)s'
        )

    def fetch_ohlcv_data(self, symbol, timeframe='5m', limit=100):
        return self.exchange.fetch_ohlcv(symbol, timeframe, limit=limit)

    def calculate_indicators(self, ohlcv_data):
        df = pd.DataFrame(
            ohlcv_data,
            columns=['timestamp', 'open', 'high', 'low', 'close', 'volume']
        )
        
        # EMA Hesaplamaları
        df['ema5'] = ta.trend.ema_indicator(df['close'], window=5)
        df['ema8'] = ta.trend.ema_indicator(df['close'], window=8)
        df['ema13'] = ta.trend.ema_indicator(df['close'], window=13)
        
        # RSI Hesaplaması
        df['rsi'] = ta.momentum.rsi(df['close'], window=14)
        
        # MACD Hesaplaması
        macd = ta.trend.MACD(df['close'], window_slow=26, window_fast=12, window_sign=9)
        df['macd'] = macd.macd()
        df['signal'] = macd.macd_signal()
        df['hist'] = macd.macd_diff()

        
        return df

    def check_btc_trend(self, btc_df):
        last_row = btc_df.iloc[-1]
        
        if (last_row['ema5'] > last_row['ema8'] and 
            last_row['ema8'] > last_row['ema13'] and 
            last_row['rsi'] > 50 and 
            last_row['macd'] > last_row['signal']):
            return 'BULLISH'
        elif (last_row['ema5'] < last_row['ema8'] and 
              last_row['ema8'] < last_row['ema13'] and 
              last_row['rsi'] < 50 and 
              last_row['macd'] < last_row['signal']):
            return 'BEARISH'
        
        return 'NEUTRAL'

    def detect_ema_crossover(self, df):
        current = df.iloc[-1]
        previous = df.iloc[-2]
        
        long_crossover = (
            
            previous['rsi'] < 30 
        )
        
        short_crossover = (
            
            previous['rsi'] > 85 
        )
        
        return 'LONG' if long_crossover else 'SHORT' if short_crossover else None

    def close_position(self, symbol):
        try:
            if symbol in self.open_positions:
                position = self.open_positions[symbol]
                current_price = self.exchange.fetch_ticker(symbol)['last']
                current_time = datetime.now()
                time_in_position = (current_time - position['entry_time']).total_seconds()
                
                # Kar/zarar ve stop loss kontrolü
                if position['type'] == 'LONG':
                    profit_percentage = (current_price - position['entry_price']) / position['entry_price'] * 100
                    stop_loss_price = position['entry_price'] * (1 - self.stop_loss_percentage / 100)
                    take_profit_price = position['entry_price'] * (1 + self.take_profit_percentage / 100)
                else:  # SHORT
                    profit_percentage = (position['entry_price'] - current_price) / position['entry_price'] * 100
                    stop_loss_price = position['entry_price'] * (1 + self.stop_loss_percentage / 100)
                    take_profit_price = position['entry_price'] * (1 - self.take_profit_percentage / 100)
                
                # Net kar/zarar hesaplaması
                trade_volume = position['volume']
                entry_cost = position['entry_price'] * trade_volume
                exit_cost = current_price * trade_volume
                
                entry_commission = entry_cost * self.commission_rate
                exit_commission = exit_cost * self.commission_rate
                
                # Kar/zarar net olarak hesaplanıyor
                if position['type'] == 'LONG':
                    profit_loss = exit_cost - entry_cost - entry_commission - exit_commission
                else:  # SHORT
                    profit_loss = entry_cost - exit_cost - entry_commission - exit_commission
                
                # Toplam kar güncelleme
                self.total_profit += profit_loss
                
                # Bakiye güncelleme
                self.current_balance += profit_loss

                if time_in_position >= self.position_timeout:
                    close_reason = "Zaman Aşımı"
                elif position['type'] == 'LONG' and current_price <= stop_loss_price:
                    close_reason = "Stop Loss"
                elif position['type'] == 'LONG' and current_price >= take_profit_price:
                    close_reason = "Kar Alma"
                elif position['type'] == 'SHORT' and current_price >= stop_loss_price:
                    close_reason = "Stop Loss"
                elif position['type'] == 'SHORT' and current_price <= take_profit_price:
                    close_reason = "Kar Alma"
                else:
                    close_reason = "Bilinmeyen Neden" 
                
                # Log kaydı
            
                logging.info(f"Pozisyon Kapatıldı: {symbol}")
                logging.info(f"Kapatma Nedeni: {close_reason}")
                logging.info(f"Giriş Fiyatı: {position['entry_price']}")
                logging.info(f"Çıkış Fiyatı: {current_price}")
                logging.info(f"Kar/Zarar Yüzdesi: {profit_percentage}%")
                logging.info(f"Kar/Zarar: {profit_loss}")
                logging.info(f"Toplam Kar: {self.total_profit}")
                
                # Pozisyonu sil
                del self.open_positions[symbol]
        
        except Exception as e:
            logging.error(f"{symbol} pozisyon kapatma hatası: {e}")

    def execute_trade(self, symbol):
        try:
            # Açık pozisyon sayısını kontrol et
            

            current_time = datetime.now()
            
            # Açık pozisyonları kontrol et ve zaman aşımı veya stop loss/kar alma olanları kapat
            positions_to_close = []
            for existing_symbol, position in list(self.open_positions.items()):
                time_in_position = (current_time - position['entry_time']).total_seconds()
                current_price = self.exchange.fetch_ticker(existing_symbol)['last']
                
                # Zaman aşımı kontrolü
                if time_in_position >= self.position_timeout:
                    positions_to_close.append(existing_symbol)
                
                # Stop loss ve kar alma kontrolü
                if position['type'] == 'LONG':
                    stop_loss_price = position['entry_price'] * (1 - self.stop_loss_percentage / 100)
                    take_profit_price = position['entry_price'] * (1 + self.take_profit_percentage / 100)
                    
                    if current_price <= stop_loss_price or current_price >= take_profit_price:
                        positions_to_close.append(existing_symbol)
                else:  # SHORT
                    stop_loss_price = position['entry_price'] * (1 + self.stop_loss_percentage / 100)
                    take_profit_price = position['entry_price'] * (1 - self.take_profit_percentage / 100)
                    
                    if current_price >= stop_loss_price or current_price <= take_profit_price:
                        positions_to_close.append(existing_symbol)
            
            # Zaman aşımına uğrayan veya stop loss/kar alma koşulunu sağlayan pozisyonları kapat
            for close_symbol in positions_to_close:
                self.close_position(close_symbol)


            # Yeni pozisyon açma kontrolü
            ticker = self.exchange.fetch_ticker(symbol)
            current_price = ticker['last']
            if len(self.open_positions) >= self.max_open_positions:
                logging.info(f"Maksimum pozisyon sayısına ulaşıldı: {len(self.open_positions)}")
                return
            # Yeni pozisyon açma
            if symbol not in self.open_positions:
                # BTC trend ve sinyal kontrolü
                btc_ohlcv = self.fetch_ohlcv_data('BTC/USDT')
                btc_df = self.calculate_indicators(btc_ohlcv)
                btc_trend = self.check_btc_trend(btc_df)
                
                ohlcv_data = self.fetch_ohlcv_data(symbol)
                df = self.calculate_indicators(ohlcv_data)
                
                signal = self.detect_ema_crossover(df)
                
                # Pozisyon açma
                if signal and (( signal == 'LONG') or 
                               (signal == 'SHORT')):
                    
                    # Sabit 100 dolarlık işlem
                    trade_volume = self.fixed_trade_amount / current_price
                    entry_commission = trade_volume * current_price * self.commission_rate
                    
                    self.open_positions[symbol] = {
                        'type': signal,
                        'entry_price': current_price,
                        'current_price': current_price,
                        'volume': trade_volume,
                        'entry_commission': entry_commission,
                        'entry_time': current_time  # Pozisyon açılış zamanı
                    }
                    
                    # Log kaydı
                    logging.info(f"Yeni Pozisyon: {symbol} - {signal}")
                    logging.info(f"Giriş Fiyatı: {current_price}")
        
        except Exception as e:
            logging.error(f"{symbol} işlem hatası: {e}")

    def run_trading_strategy(self):
        while True:
            try:
                for symbol in self.symbols:
                    self.execute_trade(symbol)
                    time.sleep(1)
                
                time.sleep(60)  # 1 dakikada bir kontrol
            
            except Exception as e:
                logging.error(f"Trading loop hatası: {e}")
                time.sleep(180)

def main():     
    exchange_id = 'binance'
    symbols = ['XRP/USDT', 'DOGE/USDT', 'SOL/USDT', 'TRX/USDT', 'PEPE/USDT', 'SHIB/USDT', 'SUI/USDT', 'HBAR/USDT', 'ADA/USDT', 'SAND/USDT', 'PNUT/USDT', 'RSR/USDT', 'RENDER/USDT', 'WLD/USDT', 'DOT/USDT', 'GALA/USDT', 'AVAX/USDT', 'LINK/USDT', 'CRV/USDT', 'XLM/USDT', 'FIL/USDT', 'NEIRO/USDT', 'ACT/USDT', 'BCH/USDT', 'ARB/USDT', 'WIF/USDT', 'NEAR/USDT', 'RUNE/USDT', 'FET/USDT', 'ETC/USDT', 'FTM/USDT', 'ENA/USDT', 'SEI/USDT', 'BONK/USDT', 'VET/USDT', 'STX/USDT', 'OP/USDT', 'COMP/USDT', 'ORDI/USDT', 'UNI/USDT', 'APT/USDT', 'ALGO/USDT', 'XVG/USDT', 'TIA/USDT', 'TAO/USDT', 'FLOKI/USDT', 'AAVE/USDT', 'BOME/USDT', 'POL/USDT', 'ENS/USDT', 'EOS/USDT',
                'MANA/USDT', 'LUMIA/USDT', 'JUP/USDT', 'AMP/USDT', 'IOTA/USDT', 'FTT/USDT', 'ICP/USDT', 'INJ/USDT', '1000SATS/USDT', 'ATOM/USDT', 'APE/USDT', 'AR/USDT', 'TON/USDT', 'ETHFI/USDT', 'CAKE/USDT', 'YFI/USDT', 'CHZ/USDT', 'EIGEN/USDT', 'THE/USDT', 'ARKM/USDT', 'TURBO/USDT', 'PENDLE/USDT', 'TRB/USDT', 'LDO/USDT', 'ZRO/USDT', 'W/USDT', 'LUNA/USDT', 'THETA/USDT', 'STRK/USDT', 'NOT/USDT', 'SNT/USDT', 'MEME/USDT', 'PEOPLE/USDT', 'DOGS/USDT', 'OM/USDT', 'NEO/USDT', 'MKR/USDT', 'EGLD/USDT', 'GRT/USDT', 'IO/USDT', 'XTZ/USDT']  # Tüm sembolleri içerecek şekilde
    
    bot = CryptoTradingBot(
        exchange_id, 
        symbols, 
        initial_balance=1000,
        test_mode=True
    )
    
    bot.run_trading_strategy()

if __name__ == "__main__":
    main()

KeyboardInterrupt: 